In [59]:
import numpy as np
import itertools as it
from PGM_PyLib.MRF import RMRFwO as mrf

In [60]:
# Function to generate random probability distributions for bounding boxes
def generate_random_probabilities(num_boxes, num_states):
    """
    Generate random probability distributions for each bounding box.
    
    Parameters:
    - num_boxes: int, number of bounding boxes
    - num_states: int, number of states (e.g., "tree", "glass", "car", "cup")

    Returns:
    - numpy array of shape (num_boxes, num_states), where each row is a
      probability distribution over the states for a bounding box.
    """
    probabilities = np.random.rand(num_boxes, num_states)  # Generate random values
    probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return probabilities

# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(n):
    """Generate an n x n symmetric matrix for 0-based indexing."""
    mat = np.random.rand(n, n)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

# Define psi and Uf functions
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix (0-based indexing).
    """
    return relational_probabilities[state_i, state_j]

# Assuming relational_probabilities is global
def Uf(rmrf, observation, row, col):
    """
    Local energy function for a bounding box in a complete graph (3 rows, 1 column RMRF).
    
    This function calculates the energy based on the current state of the bounding box
    and its relationships with all other bounding boxes (complete graph assumption).
    Additionally, it considers n-tuples (n >= 3) by averaging the weights for all such tuples.
    Observation probabilities are also factored into the energy calculation.
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (3 rows, 1 column, states should be 0-based)
    - observation: bounding box probability distributions
    - row: int, row index (0-based, corresponding to the bounding box)
    - col: int, column index (should always be 0 since it's a 1-column matrix)

    Returns:
    - float, the calculated energy for the bounding box at row `row`
    """
    current_state = rmrf[row, col]  # Get the current state of the bounding box
    energy = 0.0
    
    # 1. Factor in the observation (1-tuple) likelihood for the bounding box
    obs_prob = observation[row][current_state]  # Probability for the current state
    energy -= np.log(obs_prob)  # Add the energy contribution from the observation
    
    # 2. Calculate the pairwise (2-tuple) energy from the relationships with other bounding boxes (complete graph)
    num_boxes = rmrf.shape[0]  # Number of bounding boxes (rows in rmrf)

    for other_row in range(num_boxes):
        if other_row != row:  # Avoid self-relations
            other_state = rmrf[other_row, col]  # Get the state of the other bounding box
            # Get the potential (weight) from the relational probability matrix
            weight = relational_probabilities[current_state, other_state]
            energy -= np.log(weight)  # Add the energy contribution from the relationship

    # 3. Consider n-tuples (n >= 3) and calculate the average weight as basis for the energy
    # Iterate over all possible n-tuples (where n >= 3)
    for n in range(3, num_boxes + 1):  # Start from n=3, up to n=num_boxes
        # Get all combinations of bounding boxes of size n
        tuples = list(it.combinations(range(num_boxes), n))
        
        # For each n-tuple, calculate the average relational probability
        for t in tuples:
            # Extract the relational probabilities for all pairs in the tuple
            pairwise_weights = [
                relational_probabilities[rmrf[i, col], rmrf[j, col]]
                for i, j in it.combinations(t, 2)
            ]
            
            # Calculate the mean of the pairwise weights using NumPy
            if pairwise_weights:  # Ensure there are weights to average
                average_weight = np.mean(pairwise_weights)
                energy -= np.log(average_weight)  # Subtract the energy from the average weight
    
    return energy

In [61]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
relational_probabilities = generate_symmetric_matrix(4)
print("\nRandom Relational Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
bounding_box_probabilities = generate_random_probabilities(num_boxes=3, num_states=4)
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(bounding_box_probabilities, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, bounding_box_probabilities)  # Use the bounding box probabilities as the observation

# Run inference using the Uf function
result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")

print("\nFinal RMRF after inference (Bounding Box States):")
print(result)  # This will be a 3x1 matrix with the final state for each bounding box

Initial RMRF
 [[0]
 [0]
 [0]]

Random Relational Probability Matrix (ConceptNet):
[[1.         0.34509693 0.59663395 0.5327781 ]
 [0.34509693 1.         0.82337371 0.40749675]
 [0.59663395 0.82337371 1.         0.74500461]
 [0.5327781  0.40749675 0.74500461 1.        ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.17086678 0.31862248 0.27208753 0.23842321]
 [0.00965279 0.24383672 0.37389886 0.37261163]
 [0.37175074 0.01568511 0.32307176 0.28949239]] 

Succesfully finish, iteration: 2

Final RMRF after inference (Bounding Box States):
[[1]
 [1]
 [2]]
